In [1]:
import matplotlib.pyplot as plt
import csv
import numpy


from keras.preprocessing.image import  array_to_img, img_to_array, ImageDataGenerator


from keras.models import Sequential, model_from_json
from keras.layers import Dense, Flatten

from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras.optimizers import SGD

from PIL import Image

# Размер мини-выборки
batch_size = 32
# Количество классов изображений
nb_classes = 43
# Количество эпох для обучения
nb_epoch = 5
# Размер изображений
img_rows, img_cols = 48, 48
# Количество каналов в изображении: RGB
img_channels = 3

Using TensorFlow backend.


# функция загрузки обучающей выборки 2012 года

In [2]:
def load_img(path):
    with Image.open(path) as img:
        img = img.resize((img_rows, img_cols))
        imgarray = numpy.asarray(img)
        #mask = numpy.array([[1,1,1]])
        #imgarray = imgarray * mask
        #print(imgarray.shape)
        #imgarray = imgarray.reshape((img_rows, img_cols, img_channels))
        #print(imgarray)
        return imgarray

        
def readTrafficSigns(rootpath):
    images = [] # images
    labels = [] # corresponding labels
    # loop over all 42 classes
    for c in range(0,43):
        prefix = rootpath + '/' + format(c, '05d') + '/' # subdirectory for class
        gtFile = open(prefix + 'GT-' + format(c, '05d') + '.csv') # annotations file
        gtReader = csv.reader(gtFile, delimiter=';') # csv parser for annotations file
        gtReader.__next__() # skip header
        # loop over all images in current annotations file
        for row in gtReader:
            imgarray = load_img(prefix + row[0])
            images.append(imgarray) # the 1th column is the filename             
#             images.append(plt.imread(prefix + row[0]))
            labels.append(row[7]) # the 8th column is the label
        gtFile.close()
    return images, labels

In [3]:
trainImages, trainLabels = readTrafficSigns('./GTSRB/Training')
X_train = numpy.asarray(trainImages)/255
Y_train = np_utils.to_categorical(trainLabels, nb_classes)
#print(len(trainLabels), len(trainImages))

26640 26640


# создания  генератора

In [4]:
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=False)


datagen.fit(X_train)

# Модель

In [5]:
# Создаем последовательную модель
model = Sequential()
# Первый сверточный слой
model.add(Conv2D(100, (7, 7), padding='same', input_shape=(img_rows, img_cols, img_channels), activation='relu'))

# Первый слой подвыборки
model.add(MaxPooling2D(pool_size=(2, 2)))

# Второй сверточный слой
model.add(Conv2D(150, (4, 4), activation='relu', padding='same'))

# второй слой подвыборки
model.add(MaxPooling2D(pool_size=(2, 2)))

# Третий сверточный слой
model.add(Conv2D(250, (4, 4), padding='same', activation='relu'))

# Третий слой подвыборки
model.add(MaxPooling2D(pool_size=(2, 2)))

# Слой преобразования данных из 2D представления в плоское
model.add(Flatten())
# Полносвязный слой для классификации
model.add(Dense(300, activation='relu'))

# Выходной полносвязный слой
model.add(Dense(nb_classes, activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

# обучение с генератором

In [6]:
model.fit_generator(datagen.flow(X_train, Y_train, batch_size=batch_size),
              epochs=nb_epoch,steps_per_epoch = 1000,
              verbose=2)

Epoch 1/5
80s - loss: 1.7454 - acc: 0.4850
Epoch 2/5
77s - loss: 0.3984 - acc: 0.8778
Epoch 3/5
79s - loss: 0.2036 - acc: 0.9412
Epoch 4/5
78s - loss: 0.1486 - acc: 0.9578
Epoch 5/5
77s - loss: 0.1014 - acc: 0.9711


# дополнительное обучение на тестовых данных 2013г

In [11]:
X_13 = [] # images
Y_13 = [] # corresponding labels
for i in range(43):
    prefix = './TrainIJCNN2013/' + format(i, '02d') + '/' # subdirectory for class
    j = 0
    while True:
        try:  
            imgarray = load_img(prefix + format(j,"05d") +'.ppm')
            
            #plt.imshow(imgarray)
            #plt.show()
            X_13.append(imgarray) # the 1th column is the filename
            Y_13.append(i)
        except IOError as e:
            break
        j+=1

# print(X_test)
X_13 = numpy.asarray(X_13)
X_13 = X_13.astype('float32')
X_13 /= 255
Y_13 = np_utils.to_categorical(Y_13, nb_classes)
model.fit(X_13, Y_13,
              batch_size=batch_size,
              epochs=nb_epoch,
              validation_split=0.1,
              shuffle=True,
              verbose=2)

Train on 767 samples, validate on 86 samples
Epoch 1/5
2s - loss: 0.4012 - acc: 0.8931 - val_loss: 0.4075 - val_acc: 0.8837
Epoch 2/5
1s - loss: 0.0997 - acc: 0.9765 - val_loss: 0.4254 - val_acc: 0.8953
Epoch 3/5
1s - loss: 0.0333 - acc: 0.9935 - val_loss: 0.3161 - val_acc: 0.8837
Epoch 4/5
1s - loss: 0.0141 - acc: 0.9987 - val_loss: 0.5152 - val_acc: 0.8721
Epoch 5/5
1s - loss: 0.0055 - acc: 1.0000 - val_loss: 0.4446 - val_acc: 0.8721


# сохранение 

In [ ]:
model_json = model.to_json()
json_file = open("./all_model.json", "w")
json_file.write(model_json)
json_file.close()
model.save_weights("./all_model.h5")

In [ ]:
# загрузка
# json_file = open("./all_model.json", "r")
# loaded_model_json = json_file.read()
# json_file.close()
# # Создаем модель
# model = model_from_json(loaded_model_json)
# # Загружаем сохраненные веса в модель
# model.load_weights("./all_model.h5")
# sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
# model.compile(loss='categorical_crossentropy',
#               optimizer=sgd,
#               metrics=['accuracy'])

# тестирование

In [12]:
scores = model.evaluate(X_train, Y_train, verbose=0)
print("Точность работы загруженной сети на обучающей выборке: %.2f%%" % (scores[1]*100))

Точность работы загруженной сети на обучающей выборке: 96.69%


In [13]:
def readTrafficSigns_test(rootpath):
    images = [] # images
    labels = [] # corresponding labels
    prefix = rootpath
    gtFile = open(prefix + 'GT-final_test.csv') # annotations file
    gtReader = csv.reader(gtFile, delimiter=';') # csv parser for annotations file
    gtReader.__next__() # skip header
    # loop over all images in current annotations file
    for row in gtReader:
        imgarray = load_img(prefix + row[0])
        images.append(imgarray) # the 1th column is the filename             
#         images.append(plt.imread(prefix + row[0]))
        labels.append(row[7]) # the 8th column is the label
    gtFile.close()
    return images, labels

tImages, tLabels = readTrafficSigns_test('./GTSRB_Final_Test_Images/GTSRB/Final_Test/Images/')
X_t = numpy.asarray(tImages)/255
Y_t = np_utils.to_categorical(tLabels, nb_classes)
print(len(tLabels), len(tImages))
scores = model.evaluate(X_t, Y_t, verbose=0)
print("Точность работы загруженной сети на тестовых данных: %.2f%%" % (scores[1]*100))

12630 12630
Точность работы загруженной сети на тестовых данных: 91.53%
